# DSPy Config (MEDZ2 GPT-OSS-120B)

Minimal setup for the programming → evaluation → optimization demo.

In [ ]:
import os

# ----------------------------------------------------------------------
# MEDZ2 vLLM (OpenAI-compatible)
# IMPORTANT: base should end with /v1 for OpenAI-compatible clients
# ----------------------------------------------------------------------
MEDZ2_BASE_URL = os.getenv("MEDZ2_VLLM_BASE_URL", "http://medz2.ornl.gov:8200/v1")
MEDZ2_API_KEY = os.getenv("MEDZ2_VLLM_API_KEY", "DUMMY")

# Model id must match what /v1/models returns (exactly).
MEDZ2_MODEL = os.getenv("MEDZ2_MODEL", "openai/gpt-oss-120b")

# DSPy defaults
DSPY_TEMPERATURE = float(os.getenv("DSPY_TEMPERATURE", "0.0"))
DSPY_MAX_TOKENS = int(os.getenv("DSPY_MAX_TOKENS", "4096"))
DSPY_CACHE = os.getenv("DSPY_CACHE", "true").lower() in ("1", "true", "yes")
DSPY_NUM_RETRIES = int(os.getenv("DSPY_NUM_RETRIES", "2"))


In [ ]:
import dspy


def _make_lm():
    """Create a DSPy LM for MEDZ2 (OpenAI-compatible vLLM)."""
    if hasattr(dspy, "LM"):
        return dspy.LM(
            model=MEDZ2_MODEL,
            api_key=MEDZ2_API_KEY,
            api_base=MEDZ2_BASE_URL,
            temperature=DSPY_TEMPERATURE,
            max_tokens=DSPY_MAX_TOKENS,
            num_retries=DSPY_NUM_RETRIES,
            cache=DSPY_CACHE,
        )
    if hasattr(dspy, "OpenAI"):
        return dspy.OpenAI(
            model=MEDZ2_MODEL,
            api_key=MEDZ2_API_KEY,
            api_base=MEDZ2_BASE_URL,
            temperature=DSPY_TEMPERATURE,
            max_tokens=DSPY_MAX_TOKENS,
        )
    raise RuntimeError("Unsupported DSPy version: missing LM/OpenAI client.")


# Bootstrap once per notebook session
lm = _make_lm()
if hasattr(dspy, "settings") and hasattr(dspy.settings, "configure"):
    dspy.settings.configure(lm=lm)
elif hasattr(dspy, "configure"):
    dspy.configure(lm=lm)
else:
    raise RuntimeError("Unsupported DSPy version: missing configure API.")

print("DSPy configured with", MEDZ2_MODEL, "at", MEDZ2_BASE_URL)
